# Dev 06 - Preview

In [2]:
import os
import glob
import time
import numpy as np
import scipy.signal
import matplotlib
import matplotlib.pyplot as plt

import toshi
import cspec

zmap = cspec.colormap.zmap()
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', zmap[:, :3], N=len(zmap))

# filesize = os.path.getsize(file)

In [3]:
# Set #1 in an external hard drive
# files = []
# for day in glob.glob('/Volumes/Transcend/akita/IQ/IQdata/*'):
#     for hour in glob.glob('{}/*'.format(day)):
#         #print(day, hour)
#         for file in glob.glob('{}/*.dat'.format(hour)):
#             files.append(file)

# Set #2 from an FTP upload
files = []
homepath = os.path.expanduser('~/Downloads/20200906fromTOSHIBA/IQdata')
for file in glob.glob('{}/*.dat'.format(homepath)):
    files.append(file)

In [4]:
len(files)

9

In [6]:
for k in range(0, len(files)):
    file = files[k]
    print(file)
    
    s = time.time()
    ray_pulses, cpi_headers = toshi.read(file)
    e = time.time()
    print('Data read in {:.2f} s'.format(e - s))

    # Go through the pulses for azimuth
    a = np.zeros(len(ray_pulses), dtype=np.single)
    for k, pulses in enumerate(ray_pulses):
        a[k] = pulses[0].azimuth

    # Choose ray 2 to whatever that completes the 360-deg coverage
    n = np.argmin(np.abs(a[3:] - a[2])) + 1
    a = a[2:n+2]

    # Subsetting
    ray_pulses = ray_pulses[2:n+2]
    cpi_headers = cpi_headers[2:n+2]
    
    # Dimensions
    naz = len(ray_pulses)
    ngate_long = ray_pulses[0][0].ngate_long_hi
    ngate_short = ray_pulses[0][0].ngate_short_hi
    ngate = ngate_long + ngate_short

    # Elevation assumed to be flat from the very first pulse
    scan_el = ray_pulses[0][0].elevation
    scan_time = time.strptime(os.path.basename(file)[:15], '%Y%m%d_%H%M%S')

    # Sampling code from the CPI header
    fs = 1.0e6 * (1 << cpi_headers[0].fs_code)
    dr = 3.0e8 / fs / 2
    r = 1.0e-3 * np.arange(0, ngate, dtype=np.single) * dr

    # Noise estimate, try azimuth 0, around 20-25 km
    ia = np.argmin(np.abs(a[3:] - 0.0)) + 1
    ir, er = np.argmin(np.abs(r - 20.0)), np.argmin(np.abs(r - 25.0))
    samples = np.zeros((len(ray_pulses[ia]), er-ir), dtype=np.csingle)

    # Gather the samples. Ignore phase code since we are only interested in amplitude
    for k, pulse in enumerate(ray_pulses[ia]):
        samples[k, :] = pulse.h_long_hi[ir:er]
    noise = np.mean(np.abs(samples)) ** 2
    print('Noise estimate in 16-bit ADU: {:.4f}'.format(noise))
    
    s = np.zeros((naz, ngate), dtype=np.single)

    # Go through the pulses
    a = np.zeros(len(ray_pulses), dtype=np.single)
    for k, (pulses, cpi_header) in enumerate(zip(ray_pulses, cpi_headers)):
        p = np.zeros((len(pulses), ngate), dtype=np.csingle)
        a[k] = pulses[0].azimuth
        for j, pulse in enumerate(pulses):
            p[j, ngate_short:] = pulse.h_long_hi * np.exp(-1j * pulse.phase_h_long)
            p[j, :ngate_short] = pulse.h_short_hi * np.exp(-1j * pulse.phase_h_short)
        s[k, :] = np.mean(np.abs(p) ** 2, axis=0)

    # Signal
    s -= noise
    s[s <= 0] = 1.0e-6                                    # Avoid log(0)
    snr = 10 * np.log10(s / noise)                        # Signal-to-noise ratio
    z = 10 * np.log10(s * (r + 0.5e-3 * dr) ** 2) - 40    # Estimated ZCal = -40 
    z[:, :ngate_short] += 30                              # 30-dB on the short waveform, perhaps?

    # Thresholding
    m = snr < 0
    z[m] = np.nan
    
    # Coordinate
    ce = np.cos(np.deg2rad(scan_el))
    rr, aa = np.meshgrid(r, np.deg2rad(a))
    xx = rr * ce * np.sin(aa)
    yy = rr * ce * np.cos(aa)

    # Plot
    plt.figure(figsize=(5.2, 4), dpi=200)
    plt.pcolormesh(xx[:, :500], yy[:, :500], z[:, :500], cmap=cmap)
    # plt.plot(x_turb, y_turb, 'xk')
    plt.clim((-32, 96))
    # plt.xlim((-13, 5))
    # plt.ylim((3, 18))
    # plt.xlim((-25, 10))
    # plt.ylim((0, 35))

    plt.xlim((-20, 20))
    plt.ylim((-20, 20))

    plt.colorbar()
    plt.grid(linewidth=0.25)
    plt.title(os.path.basename(file), fontsize=8)
    im = os.path.expanduser('~/Downloads/akita/{}.png'.format(os.path.basename(file)[:-4]))
    print(im) 
    plt.savefig(im)
    plt.close()

/Users/boonleng/Downloads/20200906fromTOSHIBA/IQdata/20200814_153014.804652-33-9B.iqData.XXXX.AKITA.dat
Data read in 2.51 s
Noise estimate in 16-bit ADU: 1048.7291
/Users/boonleng/Downloads/akita/20200814_153014.804652-33-9B.iqData.XXXX.AKITA.png
/Users/boonleng/Downloads/20200906fromTOSHIBA/IQdata/20200728_040048.017743-07-9B.iqData.XXXX.AKITA.dat
Data read in 2.26 s
Noise estimate in 16-bit ADU: 20.3371
/Users/boonleng/Downloads/akita/20200728_040048.017743-07-9B.iqData.XXXX.AKITA.png
/Users/boonleng/Downloads/20200906fromTOSHIBA/IQdata/20200728_040012.511494-05-9B.iqData.XXXX.AKITA.dat
Data read in 2.07 s
Noise estimate in 16-bit ADU: 20.1591
/Users/boonleng/Downloads/akita/20200728_040012.511494-05-9B.iqData.XXXX.AKITA.png
/Users/boonleng/Downloads/20200906fromTOSHIBA/IQdata/20200809_070045.384246-D8-9B.iqData.XXXX.AKITA.dat
Data read in 2.06 s
Noise estimate in 16-bit ADU: 170.0139
/Users/boonleng/Downloads/akita/20200809_070045.384246-D8-9B.iqData.XXXX.AKITA.png
/Users/boonleng/D

Images to Movie
===

If you have ffmpeg installed, you can generate a movie from the PNG images:

```shell
ffmpeg -y -framerate 12 -f image2 -pattern_type glob -i 'akita/*.png' -vcodec libx264 -preset slow -crf 22 -b:v 12M -tune animation -pix_fmt yuv420p akita-02.mp4
```